# demo

> Demo of sample `ipyfernel` usage

In [ ]:
#| default_exp demo

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
from ipyfernel import * 

# Getting Started

Register which python executable you plan to use on the remote machine: 

In [ ]:
register_remote_kernel(remote_python='/Users/shawley/exercises/solveit/.venv/bin/python')

On the remote machine: 

1. Make sure that your SSH public key from the originating machine (generated via `ssh-keygen`) is included in the `~/.ssh/authorized_keys` file on the remote machine. 

2. Setup a TCP proxy to port 22 (the ssh port) via `bore`:
```bash
bore local 22 --to bore.pub
```
...and note the port it assigns yous, e.g. "`listening at bore.pub:3365`". 

Then in your notebook, start the remote kernel specifying that port and your username on the remote machine:

In [ ]:
port, user = 3365, 'shawley' 
start_remote(port, user)

# Progress Bar

The remote execution has the same limitation that any other local SolveIt progress bars might have, e.g. nothing with `\r` -- so no `tqdm` :-(.   Here's a simple one where the bar is a string that keeps getting longer.

In [ ]:
%%remote 
#%%remote  <-- this is the magic used for this cell
import time
import socket 
hostname = socket.gethostname()   # let's clarify that we're running remotely
print("We're executing on",hostname) 

def test_progbar():
    print("[", end="", flush=True)
    for i in range(20):
        print("=", end="", flush=True)
        time.sleep(0.15)
    print("] Done!")

test_progbar()

That worked! Chonk is the name of my remote laptop, and that progress bar went across the screen in real time over the span of a few seconds.
For other types of progress bars, see [TODO: Link to post by David or Rens...?]

# Matplotlib Plot

In [ ]:
%%remote 
#%%remote <-- cell magic for this execution (not displayed by docs)
import socket
import matplotlib.pyplot as plt
import numpy as np
hostname = socket.gethostname()   # let's make sure we're running remotely

def test_plot():
    x = np.linspace(0, 4*np.pi, 200)
    plt.figure(figsize=(8,3))
    plt.plot(x, np.sin(x), label='sin')
    plt.plot(x, np.cos(x), label='cos')
    plt.title(f"Hello from {hostname}!")
    plt.legend()
    plt.show()

test_plot()

The host name and the title confirms where it's being executed. 

# GPU Execution 

In [ ]:
%%remote 
#%%remote 
import torch 
device = 'cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu'
print("Device =",device) 

# 'Sticky' Remote Execution

Rather than using the `%%remote` magic for every cell, we can make it so that cells execute remotely by default.

In [ ]:
set_sticky()

Note: since the kernel is persistent, we don't need to import `socket` again.

In [ ]:
print("We're executing on",socket.gethostname())  # Should say 'Chonk', i.e. remote latop

The idea with sticky mode is to make it "transparent", as if you're running the notebook on the remote machine,...which includes shell commands:

In [ ]:
#| eval: false
# List the standard system apps on remote Mac laptop. 
!ls /System/Applications | head  -n 4

(The local system runs Linux BTW, which has no /System/Applications directory)

We can temporarily override sticky mode's redirect via the `%%local` IPython magic:

In [ ]:
%%local 
#%%local    <--- magic for this cell
import socket 

print("We're executing on",socket.gethostname()) 

In [ ]:
unset_sticky()

In [ ]:
# should be the same hostname as the previous local execution
print("We're executing on",socket.gethostname()) 

# Shutting down 

The remote kernel stays running, and if you don't explicitly stop it, you will eventually accumulate a whole bunch of zombie remote kernel processes, so it's good practice to remember to shut it down:

In [ ]:
stop_remote()

***

TODO: I never could get the magics to render in the docs, after many hours of trying. So all I could figure out to do was to just add them as comments.